# Medical Image - TFM
<h4>subtitle: Generación de una tubería distribuida para la extracción de características en imágenes médicas patológicas</h4>
license: European Union Public Licence (EUPL) v1.2

<table>
  <tr> <td> author name: </td> <td> Israel Llorens </td> </tr>
  <tr> <td> email: </td> <td> sanchezis@hotmail.com </td> </tr>
</table>

<h7>date: 2025/03/23</h7>

---

In [1]:
%load_ext autoreload
%autoreload

In [1]:
import init
from digital_pathology.spark import spark

25/03/24 11:22:55 WARN Utils: Your hostname, Airon.local resolves to a loopback address: 127.0.0.1; using 192.168.100.210 instead (on interface en0)
25/03/24 11:22:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/24 11:22:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/24 11:22:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/24 11:22:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/24 11:22:56 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/03/24 11:22:56 INFO SharedState: Warehouse path is 'file:/Users/illorens/Projects/source/Universidad/viu-MU-BD-and-DS-Image-Pathology/notebooks/spark-warehouse'.


In [10]:
import os
try:
    import pyspark
    from pyspark.sql.functions import col, isnan, when, count,to_date,year,month,expr,hour,dayofweek,lower,array_remove,collect_list,lit
    from pyspark.sql.functions import pandas_udf,split
    from pyspark.sql.types import StructField,StructType,StringType,DoubleType,FloatType,IntegerType, ArrayType, DoubleType, LongType
    import pyspark.sql.functions as F
except:
    pass
import pandas as pd

25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_3_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_1_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_0_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_5_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_2_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
25/03/24 11:52:56 INFO BlockManagerInfo: Removed broadcast_4_piece0 on 192.168.100.210:57096 in memory (size: 6.6 KiB, free: 9.4 GiB)
